<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>


@Copyright (C): 2010-2020, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-02-04 17:20:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-05 18:50:02   

In [ ]:
#bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
#Camera component display
import traitlets
import ipywidgets.widgets as widgets
import time
# Thread function operation library
import threading
import inspect
import ctypes

origin_widget = widgets.Image(format='jpeg', width=320, height=240)
mask_widget = widgets.Image(format='jpeg',width=320, height=240)
result_widget = widgets.Image(format='jpeg',width=320, height=240)

# create a horizontal box container to place the image widget next to eachother
image_container = widgets.HBox([origin_widget, mask_widget, result_widget])

display(image_container)

In [ ]:
#Thread related functions
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# Main process function

In [ ]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets


cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(5, 120)  #设置帧率
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0
# image.set(cv2.CAP_PROP_CONTRAST, 50)   #设置对比度 -64 - 64  2.0
# image.set(cv2.CAP_PROP_EXPOSURE, 156)  #设置曝光值 1.0 - 5000  156.0


# The default selection is red, if you want to recognize other colors, 
# please close the code in the red section, and open the code section of the other section
color_lower = np.array([0, 43, 46])
color_upper = np.array([10, 255, 255])

# #Green section
# color_lower = np.array([35, 43, 46])
# color_upper = np.array([77, 255, 255])

# # Blue section
# color_lower=np.array([100, 43, 46])
# color_upper = np.array([124, 255, 255])

# # Yellow section
# color_lower = np.array([26, 43, 46])
# color_upper = np.array([34, 255, 255])

# #Orange section
# color_lower = np.array([11, 43, 46])
# color_upper = np.array([25, 255, 255])


def Color_Recongnize():
    
    while(1):
        # get a frame and show 
        #Get the video frame and convert it to HSV format, use cvtColor () to convert the BGR format to HSV format, the parameter is cv2.COLOR_BGR2HSV.
        ret, frame = cap.read()
        #cv2.imshow('Capture', frame)
        origin_widget.value = bgr8_to_jpeg(frame)

        # change to hsv model
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # get mask Use the inRange () function and the upper and lower bounds of the blue range in the HSV model to obtain the mask. The blue part of the original video in the mask will be made white, and the other parts are black.
        mask = cv2.inRange(hsv, color_lower, color_upper)
        #cv2.imshow('Mask', mask)
        mask_widget.value = bgr8_to_jpeg(mask)

        # detect blue Bitwise and operate the mask on the original video frame, the white in the mask will be replaced with the real image.
        res = cv2.bitwise_and(frame, frame, mask=mask)
        #cv2.imshow('Result', res)
        result_widget.value = bgr8_to_jpeg(res)

        #     if cv2.waitKey(1) & 0xFF == ord('q'):
        #         break
        time.sleep(0.01)


    cap.release()
    #cv2.destroyAllWindows()

In [ ]:
#Start process
thread1 = threading.Thread(target=Color_Recongnize)
thread1.setDaemon(True)
thread1.start()

In [ ]:
#When we need to end the entire program process, we need to run the code
stop_thread(thread1)